###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Decennial SF1 2010

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4700200', #Adams city, Tennessee: Robertson
          '1600000US4702180', #Ashland City town, Tennessee: Cheatham
          '1600000US4704620', #Belle Meade city, Tennessee: Davidson
          '1600000US4705140', #Berry Hill city, Tennessee: Davidson
          '1600000US4708280', #Brentwood city, Tennessee: Williamson
          '1600000US4709880', #Burns town, Tennessee: Dickson
          '1600000US4711980', #Cedar Hill city, Tennessee: Robertson
          '1600000US4713080', #Charlotte town, Tennessee: Dickson
          '1600000US4715160', #Clarksville city, Tennessee: Montgomery
          '1600000US4716540', #Columbia city, Tennessee: Maury
          '1600000US4716980', #Coopertown town, Tennessee: Robertson
          '1600000US4718420', #Cross Plains city, Tennessee: Robertson
          '1600000US4718820', #Cumberland City town, Tennessee: Stewart
          '1600000US4720620', #Dickson city, Tennessee: Dickson
          '1600000US4721400', #Dover city, Tennessee: Stewart
          '1600000US4722360', #Eagleville city, Tennessee: Rutherford
          '1600000US4724320', #Erin city, Tennessee: Houston
          '1600000US4725440', #Fairview city, Tennessee: Williamson
          '1600000US4727020', #Forest Hills city, Tennessee: Davidson
          '1600000US4727740', #Franklin city, Tennessee: Williamson
          '1600000US4728540', #Gallatin city, Tennessee: Sumner
          '1600000US4729920', #Goodlettsville city, Tennessee: Davidson/Sumner
          '1600000US4730960', #Greenbrier town, Tennessee: Robertson
          '1600000US4733280', #Hendersonville city, Tennessee: Sumner
          '1600000US4739660', #Kingston Springs town, Tennessee: Cheatham
          '1600000US4741200', #La Vergne city, Tennessee: Rutherford
          '1600000US4740160', #Lafayette city, Tennessee: Macon
          '1600000US4741520', #Lebanon city, Tennessee: Wilson
          '1600000US4744840', #McEwen city, Tennessee: Humphreys
          '1600000US4748980', #Millersville city, Tennessee: Robertson/Sumner
          '1600000US4749460', #Mitchellville city, Tennessee: Sumner
          '1600000US4750780', #Mount Juliet city, Tennessee: Wilson
          '1600000US4751080', #Mount Pleasant city, Tennessee: Maury
          '1600000US4751560', #Murfreesboro city, Tennessee: Rutherford
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4752820', #New Johnsonville city, Tennessee: Humphreys
          '1600000US4753460', #Nolensville town, Tennessee: Williamson
          '1600000US4754780', #Oak Hill city, Tennessee: Davidson
          '1600000US4757480', #Pegram town, Tennessee: Cheatham
          '1600000US4759560', #Pleasant View city, Tennessee: Cheatham
          '1600000US4760280', #Portland city, Tennessee: Robertson/Sumner
          '1600000US4763140', #Ridgetop city, Tennessee: Davidson/Robertson
          '1600000US4769080', #Slayden town, Tennessee: Dickson
          '1600000US4769420', #Smyrna town, Tennessee: Rutherford
          '1600000US4770580', #Spring Hill city, Tennessee: Maury/Williamson
          '1600000US4770500', #Springfield city, Tennessee: Robertson
          '1600000US4773460', #Tennessee Ridge town, Tennessee: Houston/Stewart
          '1600000US4773900', #Thompson's Station town, Tennessee: Williamson
          '1600000US4776860', #Vanleer town, Tennessee: Dickson
          '1600000US4778320', #Watertown city, Tennessee: Wilson
          '1600000US4778560', #Waverly city, Tennessee: Humphreys
          '1600000US4779420', #Westmoreland town, Tennessee: Sumner
          '1600000US4779980', #White Bluff town, Tennessee: Dickson
          '1600000US4780200', #White House city, Tennessee: Robertson/Sumner
          '1600000US2128918', #Franklin city, Kentucky: 
          '1600000US2169114'] #Scottsville city, Kentucky

### SF1

In [5]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE SF12010.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [7]:
url_str= 'https://api.census.gov/data/2010/dec/sf1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [8]:
dataguide = dg1
var_list = list(dataguide['SF1 Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [9]:
one = savename

In [10]:
dataguide = dg2
var_list = list(dataguide['SF1 Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [11]:
two = savename

In [12]:
last = two

## Joining

In [13]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [14]:
data = one.merge(last, on = 'GEO_ID')

In [15]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

,,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits_mortgage,tenure_owneroccunits_nomortgage,tenure_renteroccunits,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,StateFIPS,GeoFIPS
NAME,GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Williamson County, Tennessee",0500000US47187,183182,183182,89336,6324,7997,8372,4712,2098,684,621,2155,3682,4755,6623,7440,7677,7402,6010,2131,2793,1405,1693,1930,1297,893,642,93846,6039,7718,7870,4597,1705,528,607,2188,4177,5558,7285,8076,8621,7767,6229,2141,2793,1485,1770,2202,1715,1330,1445,183182,163728,7941,396,5517,82,2784,2734,158769,8166,2.81,68498,68498,64886,3612,64886,40747,11970,12169,64886,51242,44172,7070,1734,5336,13644,11505,2139,47,187
"Wilson County, Tennessee",0500000US47189,113993,113993,55834,3746,4118,4271,2473,1480,684,534,1590,2872,3382,4097,4273,4663,4373,3739,1374,1924,1062,1406,1731,1061,566,415,58159,3572,3933,4086,2362,1273,554,543,1639,3158,3695,4283,4501,4988,4548,3864,1439,2094,1128,1498,1793,1308,970,930,113993,101379,7297,395,1276,44,1751,1851,99717,3691,2.65,45568,45568,42563,3005,42563,24602,9128,8833,42563,32177,25571,6606,1845,4761,10386,8452,1934,47,189
"Houston County, Tennessee",0500000US47083,8426,8426,4144,245,304,287,186,124,48,45,113,214,228,242,264,282,317,295,104,169,110,140,180,117,86,44,4282,228,276,298,168,92,45,45,133,202,223,268,276,307,281,329,120,165,107,141,192,162,107,117,8426,8015,193,24,25,3,35,131,7931,129,2.46,4188,4188,3349,839,3349,1365,1179,805,3349,2285,1752,533,165,368,1064,942,122,47,083
"Humphreys County, Tennessee",0500000US47085,18538,18538,9115,539,569,627,460,266,92,106,263,433,532,592,579,665,682,639,273,345,211,345,378,252,172,95,9423,502,613,596,387,213,99,77,260,471,521,595,602,681,693,720,261,380,217,308,424,331,250,222,18538,17629,463,82,35,6,101,222,17498,278,2.46,8865,8865,7454,1411,7454,3062,2561,1831,7454,5116,3887,1229,369,860,2338,1969,369,47,085
"Macon County, Tennessee",0500000US47111,22248,22248,10959,792,752,819,460,294,135,139,386,654,646,759,831,829,724,726,297,372,239,287,327,251,153,87,11289,731,727,790,496,274,135,147,356,644,679,698,819,796,785,744,280,398,246,336,406,327,222,253,22248,21364,93,78,44,3,435,231,20961,919,2.57,9861,9861,8561,1300,8561,3463,2868,2230,8561,6112,4670,1442,478,964,2449,2078,371,47,111


In [16]:
transp = data.transpose()
transp.head()

NAME,"Williamson County, Tennessee","Wilson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Macon County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Rutherford County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Nolensville town, Tennessee","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Portland city, Tennessee","Pleasant View city, Tennessee","Ridgetop city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Coopertown town, Tennessee","Columbia city, Tennessee","Cumberland City town, Tennessee","Cross Plains city, Tennessee","Dover city, Tennessee","Smyrna town, Tennessee","Slayden town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Dickson city, Tennessee","Franklin city, Kentucky","Scottsville city, Kentucky",Tennessee,United States
GEO_ID,0500000US47187,0500000US47189,0500000US47083,0500000US47085,0500000US47111,0500000US47119,0500000US47125,0500000US47149,0500000US47147,0500000US47161,0500000US47165,0500000US47169,0500000US47021,0500000US47037,0500000US47043,0500000US21213,0500000US21003,1600000US4753460,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4754780,1600000US4757480,1600000US4760280,1600000US4759560,1600000US4763140,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716980,1600000US4716540,1600000US4718820,1600000US4718420,1600000US4721400,1600000US4769420,1600000US4769080,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4720620,1600000US2128918,1600000US2169114,0400000US47,0100000US
pop,183182,113993,8426,18538,22248,80956,172331,262604,66283,13324,160645,7870,39105,626681,49666,17327,19956,5861,633,4541,2912,4529,2093,11480,4149,1874,537,37060,1468,51372,2756,4474,32588,26190,1750,6440,189,23671,4561,108755,601222,1951,395,1477,4105,2206,3206,10255,604,1324,7720,4812,62487,30278,15921,6433,314,1235,132929,4278,34681,311,1714,1417,39974,178,16440,29036,1368,2194,14538,8408,4226,6346105,308745538
agebysex_total_series,183182,113993,8426,18538,22248,80956,172331,262604,66283,13324,160645,7870,39105,626681,49666,17327,19956,5861,633,4541,2912,4529,2093,11480,4149,187

In [17]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [18]:
data = transp

In [19]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [20]:
data = data.transpose().reset_index()

In [21]:
data.head()

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits_mortgage,tenure_owneroccunits_nomortgage,tenure_renteroccunits,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,StateFIPS,GeoFIPS
0,"Williamson County, Tennessee",0500000US47187,183182.0,183182.0,89336.0,6324.0,7997.0,8372.0,4712.0,2098.0,684.0,621.0,2155.0,3682.0,4755.0,6623.0,7440.0,7677.0,7402.0,6010.0,2131.0,2793.0,1405.0,1693.0,1930.0,1297.0,893.0,642.0,93846.0,6039.0,7718.0,7870.0,4597.0,1705.0,528.0,607.0,2188.0,4177.0,5558.0,7285.0,8076.0,8621.0,7767.0,6229.0,2141.0,2793.0,1485.0,1770.0,2202.0,1715.0,1330.0,1445.0,183182.0,163728.0,7941.0,396.0,5517.0,82.0,2784.0,2734.0,158769.0,8166.0,2.81,68498.0,68498.0,64886.0,3612.0,64886.0,40747.0,11970.0,12169.0,64886.0,51242.0,44172.0,7070.0,1734.0,5336.0,13644.0,11505.0,2139.0,47.0,187.0
1,"Wilson County, Tennessee",0500000US47189,113993.0,113993.0,55834.0,3746.0,4118.0,4271.0,2473.0,1480.0,684.0,534.0,1590.0,2872.0,3382.0,4097.0,4273.0,4663.0,4373.0,3739.0,1374.0,1924.0,1062.0,1406.0,1731.0,1061.0,566.0,415.0,58159.0,3572.0,3933.0,4086.0,2362.0,1273.0,554.0,543.0,1639.0,3158.0,3695.0,4283.0,4501.0,4988.0,4548.0,3864.0,1439.0,2094.0,1128.0,1498.0,1793.0,1308.0,970.0,930.0,113993.0,101379.0,7297.0,395.0,1276.0,44.0,1751.0,1851.0,99717.0,3691.0,2.65,45568.0,45568.0,42563.0,3005.0,42563.0,24602.0,9128.0,8833.0,42563.0,32177.0,25571.0,6606.0,1845.0,4761.0,10386.0,8452.0,1934.0,47.0,189.0
2,"Houston County, Tennessee",0500000US47083,8426.0,8426.0,4144.0,245.0,304.0,287.0,186.0,124.0,48.0,45.0,113.0,214.0,228.0,242.0,264.0,282.0,317.0,295.0,104.0,169.0,110.0,140.0,180.0,117.0,86.0,44.0,4282.0,228.0,276.0,298.0,168.0,92.0,45.0,45.0,133.0,202.0,223.0,268.0,276.0,307.0,281.0,329.0,120.0,165.0,107.0,141.0,192.0,162.0,107.0,117.0,8426.0,8015.0,193.0,24.0,25.0,3.0,35.0,131.0,7931.0,129.0,2.46,4188.0,4188.0,3349.0,839.0,3349.0,1365.0,1179.0,805.0,3349.0,2285.0,1752.0,533.0,165.0,368.0,1064.0,942.0,122.0,47.0,83.0
3,"Humphreys County, Tennessee",0500000US47085,18538.0,18538.0,9115.0,539.0,569.0,627.0,460.0,266.0,92.0,106.0,263.0,433.0,532.0,592.0,579.0,665.0,682.0,639.0,273.0,345.0,211.0,345.0,378.0,252.0,172.0,95.0,9423.0,502.0,613.0,596.0,387.0,213.0,99.0,77.0,260.0,471.0,521.0,595.0,602.0,681.0,693.0,720.0,261.0,380.0,217.0,308.0,424.0,331.0,250.0,222.0,18538.0,17629.0,463.0,82.0,35.0,6.0,101.0,222.0,17498.0,278.0,2.46,8865.0,8865.0,7454.0,1411.0,7454.0,3062.0,2561.0,1831.0,7454.0,5116.0,3887.0,1229.0,369.0,860.0,2338.0,1969.0,369.0,47.0,85.0
4,"Macon County, Tennessee",0500000US47111,22248.0,22248.0,10959.0,792.0,752.0,819.0,460.0,294.0,135.0,139.0,386.0,654.0,646.0,759.0,831.0,829.0,724.0,726.0,297.0,372.0,239.0,287.0,327.0,251.0,153.0,87.0,11289.0,731.0,727.0,790.0,496.0,274.0,135.0,147.0,356.0,644.0,679.0,698.0,819.0,796.0,785.0,744.0,280

In [22]:
data.to_csv('../../Data/SF12010.csv', index = False)